In [1]:
from __future__ import print_function

import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

from keras import optimizers
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D, Flatten
from keras.layers import Dropout
from keras import backend as K

from hyperopt import Trials, STATUS_OK, tpe
from hyperopt import hp, fmin, tpe

import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.


In [2]:
x_train = np.load('X_train.npy')
y_train = np.load('Y_train.npy')
x_train, x_test, y_train, y_test = train_test_split(x_train,
                                                    y_train,
                                                    test_size=0.2)


In [3]:
def create_model(space):
    dense2 = space['dense2']
    dense3 = space['dense3']
    unfrozen_layers = space['unfrozen_layers']
    
    base_model = VGG16(include_top=False, 
                       input_tensor=None, 
                       input_shape=(48, 48, 3), 
                       pooling=None,
                       classes=10
                       )
    x = base_model.output
    x = Flatten()(x)
    x = Dense(3072, activation='relu')(x)
    x = Dense(dense2, activation='relu')(x)
    x = Dense(dense3, activation='relu')(x)
    predictions = Dense(10, activation='softmax')(x)

    model = Model(inputs=base_model.input, outputs=predictions)

    for i in range(len(base_model.layers) - unfrozen_layers):
        base_model.layers[i].trainable = False

    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy']
                  )
    model.fit(x=x_train, 
              y=y_train, 
              batch_size=32, 
              epochs=30, 
              callbacks=None, 
              validation_data=(x_test, y_test), 
              verbose=2
             )
    score, acc = model.evaluate(x_train, y_train, verbose=1)
    return -acc

In [4]:
space = {
    'dense2' : hp.choice('dense2', [512, 1024, 3072]),
    'dense3' : hp.choice('dense3', [256, 512, 1024, 3072]),
    'unfrozen_layers' : hp.choice('unfrozen_layers', [0, 1, 2, 3, 4, 5])
}


best = fmin(
    fn=create_model,
    space=space,
    algo=tpe.suggest,
    max_evals=15
)

# print (best_run)
# x_train, y_train, x_test, y_test = data()
# print("Evalutation of best performing model:")
# print(best_model.evaluate(x_test, y_test))
# print("Best performing model chosen hyper-parameters:")
# print(best_run)

Train on 33600 samples, validate on 8400 samples
Epoch 1/30
52s - loss: 0.2705 - acc: 0.9121 - val_loss: 0.1553 - val_acc: 0.9520
Epoch 2/30
51s - loss: 0.1509 - acc: 0.9519 - val_loss: 0.1191 - val_acc: 0.9650
Epoch 3/30
51s - loss: 0.1161 - acc: 0.9626 - val_loss: 0.1239 - val_acc: 0.9615
Epoch 4/30
51s - loss: 0.1032 - acc: 0.9668 - val_loss: 0.1348 - val_acc: 0.9588
Epoch 5/30
51s - loss: 0.0871 - acc: 0.9718 - val_loss: 0.1022 - val_acc: 0.9719
Epoch 6/30
51s - loss: 0.0834 - acc: 0.9725 - val_loss: 0.0940 - val_acc: 0.9733
Epoch 7/30
51s - loss: 0.0750 - acc: 0.9767 - val_loss: 0.0930 - val_acc: 0.9732
Epoch 8/30
51s - loss: 0.0690 - acc: 0.9773 - val_loss: 0.1106 - val_acc: 0.9649
Epoch 9/30
51s - loss: 0.0596 - acc: 0.9804 - val_loss: 0.1037 - val_acc: 0.9726
Epoch 10/30
52s - loss: 0.0602 - acc: 0.9805 - val_loss: 0.1052 - val_acc: 0.9733
Epoch 11/30
51s - loss: 0.0540 - acc: 0.9823 - val_loss: 0.0888 - val_acc: 0.9760
Epoch 12/30
51s - loss: 0.0504 - acc: 0.9838 - val_loss: 0

66s - loss: 0.0750 - acc: 0.9764 - val_loss: 0.1080 - val_acc: 0.9706
Epoch 8/30
66s - loss: 0.0688 - acc: 0.9775 - val_loss: 0.1369 - val_acc: 0.9636
Epoch 9/30
66s - loss: 0.0655 - acc: 0.9790 - val_loss: 0.1134 - val_acc: 0.9680
Epoch 10/30
66s - loss: 0.0589 - acc: 0.9808 - val_loss: 0.1031 - val_acc: 0.9721
Epoch 11/30
66s - loss: 0.0566 - acc: 0.9822 - val_loss: 0.0859 - val_acc: 0.9775
Epoch 12/30
66s - loss: 0.0509 - acc: 0.9832 - val_loss: 0.1024 - val_acc: 0.9737
Epoch 13/30
66s - loss: 0.0484 - acc: 0.9843 - val_loss: 0.1054 - val_acc: 0.9756
Epoch 14/30
66s - loss: 0.0486 - acc: 0.9837 - val_loss: 0.1197 - val_acc: 0.9686
Epoch 15/30
66s - loss: 0.0413 - acc: 0.9858 - val_loss: 0.0908 - val_acc: 0.9787
Epoch 16/30
66s - loss: 0.0393 - acc: 0.9869 - val_loss: 0.1020 - val_acc: 0.9732
Epoch 17/30
66s - loss: 0.0401 - acc: 0.9874 - val_loss: 0.1198 - val_acc: 0.9711
Epoch 18/30
66s - loss: 0.0440 - acc: 0.9865 - val_loss: 0.0985 - val_acc: 0.9760
Epoch 19/30
66s - loss: 0.0378

55s - loss: 0.0438 - acc: 0.9855 - val_loss: 0.0954 - val_acc: 0.9767
Epoch 15/30
55s - loss: 0.0437 - acc: 0.9863 - val_loss: 0.1054 - val_acc: 0.9743
Epoch 16/30
55s - loss: 0.0391 - acc: 0.9872 - val_loss: 0.1049 - val_acc: 0.9756
Epoch 17/30
55s - loss: 0.0391 - acc: 0.9875 - val_loss: 0.1118 - val_acc: 0.9751
Epoch 18/30
55s - loss: 0.0392 - acc: 0.9875 - val_loss: 0.1197 - val_acc: 0.9742
Epoch 19/30
55s - loss: 0.0355 - acc: 0.9885 - val_loss: 0.1044 - val_acc: 0.9783
Epoch 20/30
55s - loss: 0.0361 - acc: 0.9881 - val_loss: 0.1248 - val_acc: 0.9736
Epoch 21/30
54s - loss: 0.0312 - acc: 0.9901 - val_loss: 0.1269 - val_acc: 0.9756
Epoch 22/30
55s - loss: 0.0337 - acc: 0.9889 - val_loss: 0.1021 - val_acc: 0.9771
Epoch 23/30
55s - loss: 0.0299 - acc: 0.9907 - val_loss: 0.1313 - val_acc: 0.9727
Epoch 24/30
55s - loss: 0.0301 - acc: 0.9904 - val_loss: 0.1133 - val_acc: 0.9744
Epoch 25/30
55s - loss: 0.0292 - acc: 0.9901 - val_loss: 0.1032 - val_acc: 0.9783
Epoch 26/30
55s - loss: 0.02

60s - loss: 0.0275 - acc: 0.9926 - val_loss: 0.0821 - val_acc: 0.9844
Epoch 22/30
60s - loss: 0.0255 - acc: 0.9934 - val_loss: 0.0778 - val_acc: 0.9858
Epoch 23/30
60s - loss: 0.0255 - acc: 0.9934 - val_loss: 0.0847 - val_acc: 0.9850
Epoch 24/30
60s - loss: 0.0226 - acc: 0.9937 - val_loss: 0.0839 - val_acc: 0.9864
Epoch 25/30
60s - loss: 0.0242 - acc: 0.9937 - val_loss: 0.1844 - val_acc: 0.9699
Epoch 26/30
60s - loss: 0.0212 - acc: 0.9948 - val_loss: 0.0785 - val_acc: 0.9869
Epoch 27/30
60s - loss: 0.0231 - acc: 0.9947 - val_loss: 0.1003 - val_acc: 0.9818
Epoch 28/30
60s - loss: 0.0283 - acc: 0.9932 - val_loss: 0.0671 - val_acc: 0.9871
Epoch 29/30
60s - loss: 0.0213 - acc: 0.9938 - val_loss: 0.0783 - val_acc: 0.9857
Epoch 30/30
60s - loss: 0.0199 - acc: 0.9946 - val_loss: 0.0749 - val_acc: 0.9856
33568/33600 [============================>.] - ETA: 0sTrain on 33600 samples, validate on 8400 samples
Epoch 1/30
63s - loss: 0.2103 - acc: 0.9372 - val_loss: 0.1003 - val_acc: 0.9723
Epoch 2/

62s - loss: 0.0303 - acc: 0.9928 - val_loss: 0.0897 - val_acc: 0.9856
Epoch 29/30
62s - loss: 0.0319 - acc: 0.9928 - val_loss: 0.0713 - val_acc: 0.9885
Epoch 30/30
62s - loss: 0.0201 - acc: 0.9954 - val_loss: 0.0676 - val_acc: 0.9883
33568/33600 [============================>.] - ETA: 0sTrain on 33600 samples, validate on 8400 samples
Epoch 1/30
54s - loss: 0.2932 - acc: 0.9032 - val_loss: 0.1675 - val_acc: 0.9506
Epoch 2/30
54s - loss: 0.1551 - acc: 0.9520 - val_loss: 0.1269 - val_acc: 0.9625
Epoch 3/30
54s - loss: 0.1290 - acc: 0.9599 - val_loss: 0.1204 - val_acc: 0.9642
Epoch 4/30
54s - loss: 0.1092 - acc: 0.9654 - val_loss: 0.0981 - val_acc: 0.9708
Epoch 5/30
54s - loss: 0.0969 - acc: 0.9696 - val_loss: 0.1275 - val_acc: 0.9619
Epoch 6/30
54s - loss: 0.0837 - acc: 0.9734 - val_loss: 0.1018 - val_acc: 0.9707
Epoch 7/30
54s - loss: 0.0812 - acc: 0.9735 - val_loss: 0.1015 - val_acc: 0.9715
Epoch 8/30
54s - loss: 0.0704 - acc: 0.9780 - val_loss: 0.1060 - val_acc: 0.9706
Epoch 9/30
54s 

In [5]:
print (best)

{'unfrozen_layers': 2, 'dense3': 3, 'dense2': 0}
